In [ ]:
import os 
import test_module
from collections import namedtuple
import numpy as np
import retrieval_run
import settings
import cloud_dic
import utils
import test_module
import brewtools

In [ ]:
# instrument

fwhm=555
wavelength_range=[0.95,2.85]
ndata=1
Rfile='./examples/example_data/G570D_R_file.txt'



#retrieval_params

##gas
chemeq=0
# gaslist = ['h2o','co','tio','vo','crh','feh','na','k']
gaslist = ['h2o','ch4']
gastype_list=['U','U']
ptype=7  


cloudpath='../MieCode/'

## clouds

do_clouds=1


npatches=2
cloud_name=['powerlaw', 'grey', 'ZnS_WS15.mieff', 'tholin_WS15.mieff']
cloud_type=["deck","slab","deck","deck"]
particle_dis=['None', 'None', 'hansen',"log_normal"]

cloudpatch_index=[[1,2],[1],[1,2],[1]]



#npatches=1
#cloud_name=['MgSiO3.mieff'] #Mg2SiO4 MgSiO3
# cloud_name=['grey'] 
#cloud_type=["slab"]
#particle_dis=["hansen"] #"hansen" "log_normal"  "log_normal"

#cloudpatch_index=[[1]]


# ModelConfig:
do_fudge=1
samplemode='mcmc'

# instrument_instance = utils_dev.Instrument(fwhm,wavelength_range,ndata,Rfile)
# re_params = utils_dev.Retrieval_params(samplemode,chemeq,gaslist,gastype_list,fwhm,do_fudge,ptype,do_clouds,npatches,cloudname,cloudpacth_index,particle_dis)
# model_config_instance = utils_dev.ModelConfig(samplemode,do_fudge)
# io_config_instance = utils_dev.IOConfig()

instrument_instance = utils.Instrument(fwhm,wavelength_range,ndata,None, Rfile)
re_params = utils.Retrieval_params(samplemode,chemeq,gaslist,gastype_list,fwhm,do_fudge,ptype,do_clouds,npatches,cloud_name,cloud_type,cloudpatch_index,particle_dis, instrument_instance)
model_config_instance = utils.ModelConfig(samplemode,do_fudge,cloudpath=cloudpath)
io_config_instance = utils.IOConfig()




model_config_instance.dist= 5.84
model_config_instance.xlist ='data/gaslistRox.dat'
model_config_instance.xpath ='/Volumes/DudleyDisk/LineLists/'
model_config_instance.do_bff=1
model_config_instance.malk=0
model_config_instance.ch4=0
model_config_instance.update_dictionary()

In [ ]:
obspec = np.asfortranarray(np.loadtxt("./examples/example_data/G570D_2MHcalib.dat",dtype='d',unpack='true')) # G570D_2MassJcalib.dat
args_instance = utils.ArgsGen(re_params,model_config_instance,instrument_instance,obspec)
settings.init(args_instance)

In [ ]:
np.shape(args_instance.cloud_opaname),args_instance.cloud_opaname

In [ ]:
np.shape(args_instance.cloudmap),args_instance.cloudmap

In [ ]:
args_instance.cloudata[0,0,0],np.shape(args_instance.cloudata[0,0,0])

In [ ]:
args_instance=settings.runargs
all_params,all_params_values =utils.get_all_parametres(re_params.dictionary)

if re_params.samplemode=='mcmc':
    
    model_config_instance.ndim=len(all_params)
    model_config_instance.nwalkers=len(all_params)*16

    r2d2 = (71492e3)**2. / (model_config_instance.dist * 3.086e+16)**2.
    re_params.dictionary['refinement_params']['params']['r2d2']['distribution']=['normal',r2d2,0.1*r2d2]


    def tolerance_parameter_customized_distribution(x):
        return np.log10((np.random.rand(x)* (max(args_instance.obspec[2,:]**2)*(0.1 - 0.01))) + (0.01*min(args_instance.obspec[2,10::3]**2))) 

    for i in range(len(all_params)):
        if all_params[i].startswith('tolerance_parameter'):
            re_params.dictionary['refinement_params']['params'][all_params[i]]['distribution']=['customized',tolerance_parameter_customized_distribution]

    if model_config_instance.fresh == 0:
        p0=utils.MC_P0_gen(re_params.dictionary,model_config_instance,args_instance)

In [ ]:
params_master = namedtuple('params',all_params)
params_instance = params_master(*p0[0,:])
params_instance

In [ ]:
# params_dict = {
#     "h2o": -3.8252994551543527,
#     "ch4": -3.828340264610459,
#     "logg": 4.627822498263935,
#     "r2d2": 1.6393240071645267e-19,
#     "dlambda": -0.001430945738358047,
#     "tolerance_parameter_1": -32.39224565128183,
#     "Tint": 1191.4555892220585,
#     "alpha": 1.6060304097537386,
#     "lndelta": -1.595554828838201,
#     "T1": 1264.5049134283702,
#     "T2": 1087.7795937844917,
#     "T3": 1017.9941737643878,
#     "logp_mcd_MgSiO3": 0.9603479762757295,
#     "dp_mcd_MgSiO3": 0.145619328126494,
#     "hansan_a_mcd_MgSiO3": -1.0992518565521352,
#     "hansan_b_mcd_MgSiO3": 0.17427242916213406
# }

# theta = list(params_dict.values())
# params_master = namedtuple('params',all_params)
# params_instance = params_master(*theta)






params_dict = {
    "h2o": -4.371141335243057,
    "ch4": -4.34551135880609,
    "logg": 4.547763162396992,
    "r2d2": 1.791830696756768e-19,
    "dlambda": 0.0004015608628293697,
    "tolerance_parameter_1": -32.681422828189646,
    "Tint": 1296.0882038102566,
    "alpha": 1.5943267852101746,
    "lndelta": -0.4127530573115338,
    "T1": 1172.371088352764,
    "T2": 953.4686770379733,
    "T3": 905.2889669513706,
    "tau_mcs_MgSiO3": 10.899699379532738,
    "logp_mcs_MgSiO3": 0.8353954236470487,
    "dp_mcs_MgSiO3": 0.011037612285214942,
    "hansan_a_mcs_MgSiO3": -1.4202686369944644,
    "hansan_b_mcs_MgSiO3": 0.15977821740836615
}

theta = list(params_dict.values())
params_master = namedtuple('params',all_params)
params_instance = params_master(*theta)






# params_dict = {
#     "h2o": -4.052573460185154,
#     "ch4": -3.9686768553548184,
#     "logg": 4.519147160236017,
#     "r2d2": 1.5941449698226484e-19,
#     "dlambda": 0.001109605051720139,
#     "tolerance_parameter_1": -32.21349784423008,
#     "Tint": 1037.3776176361926,
#     "alpha": 1.3213251495857512,
#     "lndelta": 1.1810509437521763,
#     "T1": 1094.2075998969576,
#     "T2": 1674.764535149121,
#     "T3": 1245.0726138696573,
#     "tau_mcs_MgSiO3": 11.15320877699923,
#     "logp_mcs_MgSiO3": -0.4512703962536867,
#     "dp_mcs_MgSiO3": 0.004025497884733119,
#     "mu_mcs_MgSiO3": -0.22153541558649095,
#     "sigma_mcs_MgSiO3": 1.8582225937380992
# }

# theta = list(params_dict.values())
# params_master = namedtuple('params', all_params)
# params_instance = params_master(*theta)


params_instance 

In [ ]:
gnostics = 0
shiftspec, photspec, tauspec,cfunc = test_module.modelspec(params_instance,re_params,args_instance,gnostics)

In [ ]:
print(shiftspec)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(shiftspec[0,:],shiftspec[1,:])

In [ ]:
np.shape(cfunc)

In [ ]:
# This one plots a contribution function 
# get diagnostics along with the spectrum

from astropy.convolution import Gaussian1DKernel
from astropy.convolution import convolve, convolve_fft
import matplotlib.colors as colors

gnostics = 1
shiftspec, clphotspec, ophotspec,cfunc = test_module.modelspec(params_instance,args_instance.re_params,args_instance,gnostics)

nwave = args_instance.inwavenum.size
cfunc = np.reshape(cfunc[0,:,:],[cfunc.shape[1],cfunc.shape[2]])
fwhm = 0.0
wlen = shiftspec.shape[1]
wint =  shiftspec[0,0] - shiftspec[0,wlen-1]
# convolve with instrumental profile
# start by setting up kernel
# First step is finding the array index length of the FWHM
disp = wint / wlen
gwidth = int((((fwhm / disp) // 2) * 2) +1)
# needs to be odd
# now get the kernel and convolve
gauss = Gaussian1DKernel(gwidth)

for ilayer in range (0,args_instance.press.size):
    cfunc[:,ilayer] = convolve(cfunc[:,ilayer],gauss,boundary='extend')

tau1_cl_Press = convolve(clphotspec[0],gauss,boundary='extend')[::-1]
tau1_oth_Press = convolve(ophotspec[0],gauss,boundary='extend')[::-1]
    
wavenew = shiftspec[0,::-1]
press = args_instance.press.reshape(64,)
normfunc = np.zeros_like(cfunc)
for iwave in range(0,nwave):
    totcont = np.sum(cfunc[iwave,:])
    normfunc[iwave,:] = cfunc[iwave,:] / totcont

    
plt.rc('font', family='serif')
plt.rc('text', usetex=False)
fig=plt.figure(dpi=100)
# plt.axis([1.0,2.5,1000,1e-1])

ax = fig.add_subplot(1,1,1)

ax.set_yscale('log')
ax.invert_yaxis()
# ax.set_xscale('log')
#major_ticks = np.arange(1.0,15.,1.0)
#minor_ticks = np.arange(1.0,15.,0.5)
#ax.set_xticks(major_ticks)                                                       
#ax.set_xticks(minor_ticks, minor=True)                                           

plt.pcolormesh(wavenew,args_instance.press,(normfunc[::-1,:].transpose()),cmap='Greys',norm=colors.SymLogNorm(linthresh=0.001,linscale=0.00001,
                                              vmin=0., vmax=np.amax(normfunc)))

t1, = plt.plot(wavenew,(tau1_cl_Press),'m-',label=r'$\tau_{cloud} = 1.0$')
t2, = plt.plot(wavenew,(tau1_oth_Press),'c-', label =r'$\tau_{gas} = 1.0$')

plt.legend(handles=[t1,t2])

cbar = plt.colorbar(orientation='vertical',norm=colors.Normalize(clip=False),ticks=[1e-3,1e-2,0.1])
cbar.ax.set_yticklabels(['<0.1%', '1%', '10%'])
cbar.set_label('% of total', rotation=270)
plt.ylabel('Pressure / bar)')
plt.xlabel('Wavelength / $\mu m$')

# plt.savefig("/Users/fw23aao/spidernail/browndwarf_retrieval/LSR1835/LSR1835_figs/paper_figs/v2_figs/contributionF_max.pdf",format='pdf', dpi=100)
#plt.savefig(runname+'_contribution.png',format='png', dpi=120)

In [ ]:
for key, value in vars(args_instance).items():
    print(f"{key}: {value}")
